In [1]:
from types_data.utils import RasterIOInd
import geopandas as gpd, warnings

warnings.filterwarnings("ignore")
settlem_path = (
    "../Data/Raster/settlement/GHS_BUILT_C_MSZ_E2018_GLOBE_R2022A_54009_10_V1_0.tif"
)
dist_data = gpd.read_file("../Data/gjson/adm_3.geojson").sort_values(
    "area", ascending=False
)

In [2]:
sample = dist_data.head(2)

In [8]:
ghs = RasterIOInd(
    settlem_path,
    dist_data,
    "ghs",
    only_metric=True,
    settlement=True,
)
ghs.get_result()


100%|██████████| 1141/1141 [10:37<00:00,  1.79it/s]


In [5]:
# ghs.result_df

In [15]:
result = ghs.result_df.fillna(method="ffill")
result.to_csv("../output/GHS/base/ghs.csv", index = False) 
ghs.result_df.to_csv("../output/GHS/base/ghs_native.csv", index = False)

# Package

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
result = pd.read_csv("../output/GHS/base/ghs.csv")

In [2]:
from types_data.settlement import GHS
results = GHS(result, save_dir="../output/GHS").generate_data()

In [3]:
results.with_na_result.head()

,index,id,fnid,parent_id,admin_0,admin_1,admin_2,admin_3,ghs_1,ghs_11,...,ghs_2,ghs_21,ghs_22,ghs_23,ghs_24,ghs_25,ghs_255,ghs_3,ghs_4,ghs_5
0,112.0,222908.0,ET2023A3020207,222703.0,Ethiopia,Afar,Kilbati,Afdera,0.024064,0.019431,...,0.000070,0.0,0.0,0.0,0.0,0.0,99.956237,0.000001,0.0,0.000197
6,761.0,223557.0,ET2023A3050503,222748.0,Ethiopia,Somali,Korahe,Shilabo,0.062649,0.058097,...,0.015415,0.0,0.0,0.0,0.0,0.0,99.857949,0.000437,0.0,0.000073
14,797.0,223593.0,ET2023A3050901,222752.0,Ethiopia,Somali,Liben,Filtu,0.013929,0.032652,...,0.029799,0.0,0.0,0.0,0.0,0.0,99.916835,0.002658,0.0,0.000384
22,94.0,222890.0,ET2023A3020102,222702.0,Ethiopia,Afar,Awusi,Elidar,0.066136,0.054900,...,0.000116,0.0,0.0,0.0,0.0,0.0,99.875394,0.000037,0.0,0.001524
30,742.0,223538.0,ET2023A3050304,222746.0,Ethiopia,Somali,Jarar,Gashamo,0.092533,0.107168,...,0.037428,0.0,0.0,0.0,0.0,0.0,99.759242,0.002039,0.0,0.000316


In [4]:
results.without_na_result.head()

,index,id,fnid,parent_id,admin_0,admin_1,admin_2,admin_3,ghs_1,ghs_11,...,ghs_15,ghs_2,ghs_21,ghs_22,ghs_23,ghs_24,ghs_25,ghs_3,ghs_4,ghs_5
0,112.0,222908.0,ET2023A3020207,222703.0,Ethiopia,Afar,Kilbati,Afdera,54.987897,44.399829,...,0.0,0.159476,0.0,0.0,0.0,0.0,0.0,0.002848,0.0,0.449950
6,761.0,223557.0,ET2023A3050503,222748.0,Ethiopia,Somali,Korahe,Shilabo,44.103046,40.898834,...,0.0,10.851448,0.0,0.0,0.0,0.0,0.0,0.307634,0.0,0.051147
14,797.0,223593.0,ET2023A3050901,222752.0,Ethiopia,Somali,Liben,Filtu,16.748920,39.261501,...,0.0,35.831135,0.0,0.0,0.0,0.0,0.0,3.195741,0.0,0.461158
22,94.0,222890.0,ET2023A3020102,222702.0,Ethiopia,Afar,Awusi,Elidar,53.075960,44.059348,...,0.0,0.093065,0.0,0.0,0.0,0.0,0.0,0.029952,0.0,1.222682
30,742.0,223538.0,ET2023A3050304,222746.0,Ethiopia,Somali,Jarar,Gashamo,38.433941,44.512805,...,0.0,15.546032,0.0,0.0,0.0,0.0,0.0,0.846722,0.0,0.131063
